<a href="https://colab.research.google.com/github/rafiatulzannah/Amazon-customer-review-sentiment-analysis-using-fine-tuned-BERT/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text

In [ ]:
# Import libraries
import ast
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm
import nltk
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
plt.style.use('ggplot')
nltk.download('all')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import re
!pip install scikit-learn

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/final_dataset_v4.csv')

In [ ]:
df['recommended_v1'] = df['recommended'].apply(lambda i:1 if i == 'yes' else 0)
df['reviews'] = [ ast.literal_eval(x) for x in df['text_lemmatized']]
df['review'] = [ ' '.join(x) for x in df['reviews']]

In [ ]:
df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect = TfidfVectorizer()
X = vect.fit_transform(df['review'])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, df['recommended_v1'], stratify=df['recommended_v1'])

In [ ]:
# for machine model import required libraries

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# logistic Regression Model

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred= logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred,y_test)
print("Test Accuracy: {:.2f}% ".format(logreg_acc*100))

Test Accuracy: 87.12% 


In [ ]:
#confusion Metrics for Logistic Regression

print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

[[1520  694]
 [ 264 4961]]


              precision    recall  f1-score   support

           0       0.85      0.69      0.76      2214
           1       0.88      0.95      0.91      5225

    accuracy                           0.87      7439
   macro avg       0.86      0.82      0.84      7439
weighted avg       0.87      0.87      0.87      7439



In [ ]:
#multinomial naive bayes model

mnultinb = MultinomialNB()
mnultinb.fit(x_train, y_train)
mnultinb_pred= mnultinb.predict(x_test)
mnultinb_acc = accuracy_score(mnultinb_pred,y_test)
print("Test Accuracy: {:.2f}% ".format(mnultinb_acc*100))

Test Accuracy: 80.10% 


In [ ]:
#confusion Metrics for multinomial naive bayes model

print(confusion_matrix(y_test, mnultinb_pred))
print("\n")
print(classification_report(y_test, mnultinb_pred))

[[ 809 1405]
 [  75 5150]]


              precision    recall  f1-score   support

           0       0.92      0.37      0.52      2214
           1       0.79      0.99      0.87      5225

    accuracy                           0.80      7439
   macro avg       0.85      0.68      0.70      7439
weighted avg       0.82      0.80      0.77      7439



In [ ]:
#Linear SVC model
lnrsvc = LinearSVC()
lnrsvc.fit(x_train, y_train)
lnrsvc_pred= lnrsvc.predict(x_test)
lnrsvc_acc = accuracy_score(lnrsvc_pred,y_test)
print("Test Accuracy: {:.2f}% ".format(lnrsvc_acc*100))

Test Accuracy: 86.83% 


In [ ]:
from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import svm
clf2 = svm.SVC(kernel='linear', C=1,probability=True)

In [ ]:
#confusion Metrix for Linear SVC model

print(confusion_matrix(y_test, lnrsvc_pred))
print("\n")
print(classification_report(y_test, lnrsvc_pred))

[[1595  619]
 [ 361 4864]]


              precision    recall  f1-score   support

           0       0.82      0.72      0.76      2214
           1       0.89      0.93      0.91      5225

    accuracy                           0.87      7439
   macro avg       0.85      0.83      0.84      7439
weighted avg       0.87      0.87      0.87      7439



In [ ]:
#Fitting K-NN classifier to the training set
from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )
classifier.fit(x_train, y_train)
classifier_pred= classifier.predict(x_test)
classifier_acc = accuracy_score(classifier_pred,y_test)
print("Test Accuracy: {:.2f}% ".format(classifier_acc*100))

Test Accuracy: 71.46% 


In [ ]:
#confusion Metrix for Fitting K-NN classifier

print(confusion_matrix(y_test, classifier_pred))
print("\n")
print(classification_report(y_test, classifier_pred))

[[ 122 2092]
 [  31 5194]]


              precision    recall  f1-score   support

           0       0.80      0.06      0.10      2214
           1       0.71      0.99      0.83      5225

    accuracy                           0.71      7439
   macro avg       0.76      0.52      0.47      7439
weighted avg       0.74      0.71      0.61      7439



In [ ]:
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
class_names=[0,1]
c=make_pipeline(vect,classifier)
explainer=LimeTextExplainer(class_names=class_names)

In [ ]:
df3=pd.DataFrame()
original_text = vect.inverse_transform(x_test)
df3['data'] = [ ' '.join(x) for x in original_text]

In [ ]:
exp = explainer.explain_instance(df3['data'][3], classifier_fn = c.predict_proba, num_features=10)

In [ ]:
#Fitting Decision Tree classifier to the training set
from sklearn.ensemble import RandomForestClassifier
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")
classifier.fit(x_train, y_train)
classifier_pred= classifier.predict(x_test)
classifier_acc = accuracy_score(classifier_pred,y_test)
print("Test Accuracy: {:.2f}% ".format(classifier_acc*100))

Test Accuracy: 82.18% 


In [ ]:
#confusion Metrix for Fitting Decision Tree

print(confusion_matrix(y_test, classifier_pred))
print("\n")
print(classification_report(y_test, classifier_pred))

[[1394  820]
 [ 506 4719]]


              precision    recall  f1-score   support

           0       0.73      0.63      0.68      2214
           1       0.85      0.90      0.88      5225

    accuracy                           0.82      7439
   macro avg       0.79      0.77      0.78      7439
weighted avg       0.82      0.82      0.82      7439



In [ ]:
# grid search cv for trying to improving the accuracy

from sklearn.model_selection import GridSearchCV
param_grid = {'C' : [0.1, 1, 10, 100], 'loss': ['hinge', 'squared_hinge']}
grid = GridSearchCV(lnrsvc, param_grid, refit = True, verbose = 3)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .................C=0.1, loss=hinge;, score=0.837 total time=   0.1s
[CV 2/5] END .................C=0.1, loss=hinge;, score=0.829 total time=   0.1s
[CV 3/5] END .................C=0.1, loss=hinge;, score=0.832 total time=   0.1s
[CV 4/5] END .................C=0.1, loss=hinge;, score=0.831 total time=   0.2s
[CV 5/5] END .................C=0.1, loss=hinge;, score=0.833 total time=   0.2s
[CV 1/5] END .........C=0.1, loss=squared_hinge;, score=0.871 total time=   0.2s
[CV 2/5] END .........C=0.1, loss=squared_hinge;, score=0.861 total time=   0.3s
[CV 3/5] END .........C=0.1, loss=squared_hinge;, score=0.866 total time=   0.1s
[CV 4/5] END .........C=0.1, loss=squared_hinge;, score=0.865 total time=   0.1s
[CV 5/5] END .........C=0.1, loss=squared_hinge;, score=0.869 total time=   0.0s
[CV 1/5] END ...................C=1, loss=hinge;, score=0.874 total time=   0.3s
[CV 2/5] END ...................C=1, loss=hinge;,

GridSearchCV(estimator=LinearSVC(C=1, loss='hinge'),
             param_grid={'C': [0.1, 1, 10, 100],
                         'loss': ['hinge', 'squared_hinge']},
             verbose=3)

In [ ]:
print("Best Cross Validation Score: {:.2f}".format(grid.best_score_))
print("Best Parameters: ", grid.best_params_)

Best Cross Validation Score: 0.87
Best Parameters:  {'C': 1, 'loss': 'hinge'}


In [ ]:
#Linear SVC model (Tuned)

lnrsvc = LinearSVC(C = 1, loss = 'hinge')
lnrsvc.fit(x_train, y_train)
lnrsvc_pred= lnrsvc.predict(x_test)
lnrsvc_acc = accuracy_score(lnrsvc_pred,y_test)
print("Test Accuracy: {:.2f}% ".format(lnrsvc_acc*100))

Test Accuracy: 87.27% 


In [ ]:
#confusion Metrix for Linear SVC model (Tuned)

print(confusion_matrix(y_test, lnrsvc_pred))
print("\n")
print(classification_report(y_test, lnrsvc_pred))

[[1574  640]
 [ 307 4918]]


              precision    recall  f1-score   support

           0       0.84      0.71      0.77      2214
           1       0.88      0.94      0.91      5225

    accuracy                           0.87      7439
   macro avg       0.86      0.83      0.84      7439
weighted avg       0.87      0.87      0.87      7439

